In [1]:
# Import Libaries and Tools
import os
import sys
from astropy.io import fits
import sunpy.map
from aiapy.calibrate import register, update_pointing
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
import glob

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_fall_2023 import acweSaveSeg_v5, acweRestoreScale
from DatasetTools import DataManagmentTools as dmt

plt.rcParams.update({'image.interpolation': 'None'})

In [ ]:
# Dataset

# Carrington Rotation
CR = 'CR2101'

# ACWE Parameters 
acweChoice = '193'

# Dataset
dataset = '/home/jgra/Coronal Holes/newDataset/'

# ACWE Folders
folderBase = '/mnt/coronal_holes/Paper 2/Code 02 Observations/HMI_Experiments/'
bigACWE = folderBase + '/TestEvolutionMethods/Scaled/Uni/'
oldACWE = folderBase + '/TestEvolutionMethods/Standard/Uni/'
newACWE = folderBase + '/TestEvolutionMethods/SeedTransfer/Uni/'

# Stats file
oldDatasetFile = '/home/jgra/Coronal Holes/2nd paper/Code 02/HMI_Experiments/'
oldDatasetFile = oldDatasetFile + 'TestEvolutionMethods/Scaled/Analysis/Results/'
oldDatasetFile = oldDatasetFile + CR + '.csv.Uni.ScaleStats.npz'
NewDatasetFile = 'Results/'+CR+'.csv.Uni.SeedTransfer.npz'

In [ ]:
# Open File and Prepare for ACWE
def openAIA(filename):
    
    # Extract Image and Header Data
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') # no clue why this is needed for successful data read
    h = hdulist[1].header
    J = hdulist[1].data
    hdulist.close()
    
    # Update to Level 1.5 Data Product
    if h['LVL_NUM'] < 1.5:
        m = sunpy.map.Map((J,h))    # Create Sunpy Map
        m = update_pointing(m)      # Update Header based on Latest Information
        m_registrered = register(m) # Recenter and rotate to Solar North
        I = m_registrered.data
        # Undo Keword Renaming
        H = dict()
        for k in m_registrered.meta.keys(): 
            H[k.upper()] = m_registrered.meta[k] 
    # Skip if already Level 1.5
    else:
        I = J*1
        H = dict(h)
        
        # Prepare Display Version
    Idsp = np.clip(I,20,2500)
    Idsp = np.log10(Idsp)
    Idsp = Idsp - np.min(Idsp)
    Idsp = Idsp/np.max(Idsp)
    
    return I,Idsp,H

In [ ]:
# Get Stats

# Open and Retreave Old Data
data    = np.load(oldDatasetFile,allow_pickle=True)
lst     = data.files
oldIOUs = data[lst[1]]

# Open and Retreave New Data
data    = np.load(NewDatasetFile, allow_pickle=True)
lst     = data.files
newIOUs = data[lst[1]]

In [ ]:
# Get Images and Segmentations

# Get List of Images
euvImages = sorted(glob.glob(dataset + CR + '/*/*.' + acweChoice + '.*'))

# Get List of Segmentaitons
bigACWEsegs = sorted(glob.glob(bigACWE + CR + '/*/*.' + acweChoice + '.*'))
oldACWEsegs = sorted(glob.glob(oldACWE + CR + '/*/*.' + acweChoice + '.*'))
newACWEsegs = sorted(glob.glob(newACWE + CR + '/*/*.' + acweChoice + '.*'))

In [ ]:
# Cycle Through Dataset

for i in range(len(oldIOUs)):
    
    # Check IOU
    if oldIOUs[i,0,1] <= 0.5:
        
        # Get Corrispoinding Data
        oldIOU     = oldIOUs[i,0,1]
        newIOU     = newIOUs[i,0,1]
        euvImage   = euvImages[i]
        bigACWEseg = bigACWEsegs[i]
        oldACWEseg = oldACWEsegs[i]
        newACWEseg = newACWEsegs[i]
        
        # Open Image
        _,Idsp,_ = openAIA(euvImage)
        
        # Open Segmentations
        _,_,bigSeg     = acweSaveSeg_v5.openSeg(bigACWEseg)
        _,oldAH,oldSeg = acweSaveSeg_v5.openSeg(oldACWEseg)
        _,newAH,newSeg = acweSaveSeg_v5.openSeg(newACWEseg)
        
        # Resize Segmentations
        oldSeg = acweRestoreScale.upscale(oldSeg,oldAH)
        newSeg = acweRestoreScale.upscale(newSeg,newAH)
        
        # Display Results
        plt.figure(figsize=[14,23],dpi=300)
        plt.rcParams.update({'font.size': 22})
        title = os.path.basename(euvImage) + '\nold IOU:' + str(oldIOU) + '\nnew IOU:' + str(newIOU)
        plt.suptitle(title)
        plt.subplot(3,2,1)
        plt.imshow(np.flip(Idsp,axis=0),cmap='gray')
        plt.title('EUV Observation')
        plt.axis(False)
        plt.subplot(3,2,2)
        plt.imshow(np.flip(bigSeg,axis=0))
        plt.title('Full Scale')
        plt.axis(False)
        plt.subplot(3,2,3)
        plt.imshow(np.flip(oldSeg,axis=0))
        plt.title('1/8 Scale')
        plt.axis(False)
        plt.subplot(3,2,4)
        plt.imshow(np.flip(oldSeg.astype(int)-bigSeg.astype(int),axis=0),cmap='gray',vmin=-1,vmax=1)
        plt.title('1/8 - Full')
        plt.axis(False)
        plt.subplot(3,2,5)
        plt.imshow(np.flip(newSeg,axis=0))
        plt.title('1/8 Using Full-Size Seed')
        plt.axis(False)
        plt.subplot(3,2,6)
        plt.imshow(np.flip(newSeg.astype(int)-bigSeg.astype(int),axis=0),cmap='gray',vmin=-1,vmax=1)
        plt.title('Full-Size Seed - Full')
        plt.axis(False)
        plt.show()

In [ ]:
# Cycle Through Dataset (Reverse)

for i in range(len(newIOUs)):
    
    # Check IOU
    if newIOUs[i,0,1] <= 0.5:
        
        # Get Corrispoinding Data
        oldIOU     = oldIOUs[i,0,1]
        newIOU     = newIOUs[i,0,1]
        euvImage   = euvImages[i]
        bigACWEseg = bigACWEsegs[i]
        oldACWEseg = oldACWEsegs[i]
        newACWEseg = newACWEsegs[i]
        
        # Open Image
        _,Idsp,_ = openAIA(euvImage)
        
        # Open Segmentations
        _,_,bigSeg     = acweSaveSeg_v5.openSeg(bigACWEseg)
        _,oldAH,oldSeg = acweSaveSeg_v5.openSeg(oldACWEseg)
        _,newAH,newSeg = acweSaveSeg_v5.openSeg(newACWEseg)
        
        # Resize Segmentations
        oldSeg = acweRestoreScale.upscale(oldSeg,oldAH)
        newSeg = acweRestoreScale.upscale(newSeg,newAH)
        
        # Display Results
        plt.figure(figsize=[14,23],dpi=300)
        plt.rcParams.update({'font.size': 22})
        title = os.path.basename(euvImage) + '\nold IOU:' + str(oldIOU) + '\nnew IOU:' + str(newIOU)
        plt.suptitle(title)
        plt.subplot(3,2,1)
        plt.imshow(np.flip(Idsp,axis=0),cmap='gray')
        plt.title('EUV Observation')
        plt.axis(False)
        plt.subplot(3,2,2)
        plt.imshow(np.flip(bigSeg,axis=0))
        plt.title('Full Scale')
        plt.axis(False)
        plt.subplot(3,2,3)
        plt.imshow(np.flip(oldSeg,axis=0))
        plt.title('1/8 Scale')
        plt.axis(False)
        plt.subplot(3,2,4)
        plt.imshow(np.flip(oldSeg.astype(int)-bigSeg.astype(int),axis=0),cmap='gray',vmin=-1,vmax=1)
        plt.title('1/8 - Full')
        plt.axis(False)
        plt.subplot(3,2,5)
        plt.imshow(np.flip(newSeg,axis=0))
        plt.title('1/8 Using Full-Size Seed')
        plt.axis(False)
        plt.subplot(3,2,6)
        plt.imshow(np.flip(newSeg.astype(int)-bigSeg.astype(int),axis=0),cmap='gray',vmin=-1,vmax=1)
        plt.title('Full-Size Seed - Full')
        plt.axis(False)
        plt.show()